# Présentation de la boîte à outil Julia pour l'algèbre Max-Plus

## Algèbre tropicale

L'algebre Max-Plus (max, +) redéfinit les opérateurs addition et multiplication de l'algèbre classique par respectivement les opérateurs maximum et addition dans le domaine des nombres réels $\mathbb{R}$ augmenté du nombre moins l'infini ($\varepsilon = -\infty$) que l'on nommera par la suite $\mathbb{R}_{\varepsilon} = \mathbb{R} \cup \{ -\infty \}$. Sa structure algébrique est un dioïde $(\mathbb{R}_{\varepsilon}, \oplus, \otimes)$.

$$\forall a,b \in \mathbb{R}_{\varepsilon}: a \oplus b \triangleq \max(a,b)$$
$$\forall a,b \in \mathbb{R}_{\varepsilon}: a \otimes b \triangleq a + b$$

L'algebre Min-Plus (min, +), quand à elle, redéfinit les opérateurs addition et multiplication par les opérateurs minimum et addition. Max-Plus et Min-Plus font partie des mathématiques dites *tropicales*. L'algèbre Max-Plus permet de modéliser et d'évaluer les performances de systèmes à évènements discrets (réseaux de transport ou de télécom, systèmes de production), mais également dans des domaines tels que la théorie de la décision, recherche opérationnelle ...

Une boîte à outil Max-Plus pour le langage Julia peut être téléchargée depuis ce repo GitHub https://github.com/Lecrapouille/MaxPlus.jl ou bien depuis le système de paquets de Julia via la commande `] add MaxPlus`. Ce projet est un portage de la boîte à outil qui état intégrée dans le logiciel Scilab, puis par la suite, dans le logiciel [ScicosLab](http://www.scicoslab.org), un fork maintenu à l'époque par les anciens chercheurs de Scilab mais qui est désormais remplacé par le second fork [NSP](https://cermics.enpc.fr/~jpc/nsp-tiddly/mine.html).

Ce document présente les fonctions de base de cette boîte à outils Julia tout en introduisant les bases de l'algèbre Max-Plus. Les lecteurs maîtrisant déjà la théorie de cette algèbre, peuvent aller directement consulter cette [bibliographie](../docs/src/bibliography.md). Pour ceux qui désirent comparer les résultats obtenus avec ceux fournis par Sicoslab, on rappellera qu'un nombre Max-Plus Sicoslab est créé avec la notation suivante `#(42)`, que les éléments neutres et absorbants obtenus par `%0` et `%1`, qu'une démonstration interactive de la boîte à outil peut être lancée depuis leur menu et que le document PDF [suivant](https://jpquadrat.github.io/TPALGLIN.pdf) en fournit un résumé.

## Installation de la boîte à outil Max-Plus Julia

Tout d'abord vérifions la version de votre Julia. Cette boîte à outils Max-Plus doit fonctionner avec toutes les versions de Julia même celles obsolètes (v0.4, v0.7). Certaines versions de Julia apportent des correctifs (comme sur les matrices creuses: v1.3), d'autres ajoutent des régressions (v1.0: matrice identité; v1.4: produit matriciel matrice creuse avec matrice pleine; v1.5: affichage matrice creuses comme des matrices denses). D'autres bugs sont vieux et ne sont pas corrigés mais des correctifs sont automatiquement appliqués avec ce paquet Max-Plus. Pour plus d'information, confère le fichier `fallbacks.jl`.

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD Ryzen 7 1800X Eight-Core Processor
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, znver1)


Ensuite, nous allons installer la boîte à outils Max-Plus Julia. Vous avez plusieurs options pour le faire. Les codes suivants ne fonctionnent pas directement depuis ce document Jupyter, pensez donc à les décommenter et de les exécuter depuis le mode interactif Julia (REPL) :
- Vous pouvez télécharger la version du code source depuis GitHub :

In [2]:
# using Pkg; Pkg.add(PackageSpec(url="https://github.com/Lecrapouille/MaxPlus.jl"));
# using MaxPlus;

- Soit depuis le gestionnaire de paquet Julia (version stable):

In [3]:
# using Pkg; Pkg.add("MaxPlus")

## Utiliser Max-Plus depuis un document Jupyter

Maintenant que le paquet Julia Max-Plus a été installé, vous pouvez directement utiliser les nombres et fonctions Max-Plus depuis le REPL de Julia. Comme c'est encore plus agréable de les taper depuis un document Jupyter, on va lancer, depuis le REPL, un notebook Jupyter et ensuite ouvrir cette précente page :

In [4]:
# using IJulia
# notebook()

Depuis ce document Jupyter (celui que vous lisez), chargez la boîte à outil Max-Plus depuis le dossier MaxPlus.jl :

In [5]:
push!(LOAD_PATH, pwd())
using MaxPlus

┌ Info: Precompiling MaxPlus [41177cfe-c387-11e9-2806-edd030e4594e]
└ @ Base loading.jl:1317


Pour le moment et dans un soucis pédagogique, on active un mode d'affichage particulier des nombres Max-Plus pour forcer l'affichage des nombres $-\infty$. L'explication sera donnée plus tard.

In [6]:
mp_change_display(0);

Cette boîte à outils permet de générer du code $\LaTeX$ via la fonction `Base.show`. Sur Jupyter ce mode semble être forcé mais on veut garder l'affichage en texte plein.

In [7]:
Base.show(io::IO, ::MIME"text/latex", x::MP) = show(io, MIME"text/plain", x)
Base.show(io::IO, ::MIME"text/latex", A::ArrMP) = show(io, MIME"text/plain", A)
Base.show(io::IO, ::MIME"text/latex", S::SpaMP) = show(io, MIME"text/plain", S)

## Scalaires Max-Plus pour Julia

Avant de présenter l'algèbre Max-Plus, créons quelques nombres scalaires Max-Plus sous Julia. Par exemple, on écrira :

In [8]:
a = MP(1.0);  b = MP(3.5);  c = MP(5)
typeof(a), typeof(b), typeof(c)

(MP, MP, MP)

Les nombres Max-Plus sont codés en interne par des `Float64` car ils sont seulement définis dans l'espace $\mathbb{R}_{\varepsilon}$. On peut directement accéder à la valeur via le champ `λ` pour repasser dans l'algébre classique.

In [9]:
a,   a.λ,   typeof(a),   typeof(a.λ)

(1.0, 1.0, MP, Float64)

In [10]:
c,   c.λ,   typeof(c),   typeof(c.λ)

(5.0, 5.0, MP, Float64)

Si l'on ne desire pas utiliser explicitement le champ `λ` il existe une fonction `plustimes` pour le faire qui fonctionnera également pour les matrices creuses et pleines. Le scalaire (ou la matrice) retourné fonctionnera dans l'algébre classique (addition et multiplication classique) :

In [11]:
a,   plustimes(a),   typeof(a),   typeof(plustimes(a))

(1.0, 1.0, MP, Float64)

In [12]:
a + a,   plustimes(a) + plustimes(a)

(1.0, 2.0)

Les nombres Max-Plus contaminent sur les autres nombres (entiers, réels) : ils convertissent un nombre non Max-Plus en nombre Max-Plus via les opérateurs arithmétiques où les opérateurs de promotion implicites :

In [13]:
d = 1.0
typeof(d),   typeof(c),   typeof(c + d),   typeof((c + d).λ),   c + d

(Float64, MP, MP, Float64, 5.0)

Nous voyons que l'addition Max-Plus a converti `d` de type `Float64` en type `MP`. Même comportement pour les nombres entiers où `f` de type `Int64` est converti en en type `MP` :

In [14]:
f = 1
typeof(f), typeof(c), typeof(c + f), typeof((c + f).λ),   c + f

(Int64, MP, MP, Float64, 5.0)

## Les constantes Max-Plus pour Julia

Les éléments neutres ou absorbants pour les opérateurs $\oplus$ et $\otimes$ sont donnés sous forme de constantes Julia :
- Elément neutre $\varepsilon$ (parfois noté $\mathbb{0}$ dans certains documents) pour les opérateurs $\oplus$ et $\otimes$ : les constants Julia sont `mp0` ou bien `ε` (obtenu en tapant `\varepsilon`) vallant toutes les deux $-\infty$.
- Elément neutre $e$ (parfois noté $\mathbb{1}$ dans certains documents) pour l'opérateur $\otimes$ : les constants Julia sont
`mp1` ou bien `mpe` vallant toutes les deux 0.
- Bien que cette boite à outil se focalise sur l'alègbre (max, +) elle offre la constante `mptop` vallant $+\infty$ utilisée dans l'algèbre (min, +).

Ces nombres sont de type `MP` (et l'on pourra éventuellement les convertir en nombre de l'algèbre classique soit via le champ `λ` soit la fonction `plustimes`.

In [15]:
mp0, ε, mp1, mpe, mptop, mpzero(), mpone(), mptop

(-Inf, -Inf, 0.0, 0.0, Inf, -Inf, 0.0, Inf)

Notons que `ε` peut être converti en Min-Plus ou Max-Plus via la fonction `minplus`. Elle inverse simplement le signe pour `ε` mais, comme on le verra plus tard, elle s'applique sur tous les éléments des matrices creuses et pleines.

In [16]:
minplus(ε),   minplus(minplus(ε))

(Inf, -Inf)

In [17]:
minplus(ε) == mptop,   minplus(minplus(ε)) == ε

(true, true)

## Contrôle de l'affichage des nombres Max-Plus

Nous voyons que Julia affiche `-Inf` pour `ε` ce qui n'est pas très compact. Il y a 4 styles possibles d'affichage des nombres Max-Plus que l'on peut contrôler avec la fonction `mp_change_display`, le mode `1` étant celui défini par défaut et suit l'affichage dans ScicosLab :
- Style 0: les nombres $-\infty$ et les 0 sont affichés tels quels.
- Style 1 ou 2: les nombres $-\infty$ sont affiché sous la forme d'un point `.`.
- Style 3 ou 4: les nombres $-\infty$ sont affichés sous la forme $\varepsilon$.
- Style 1 ou 3: les zéros sont affichés 0.
- Style 2 ou 4: les zéros sont affichés $e$.

Notons que les nombres réels qui peuvent être écrits comme des entiers (donc sans chiffres apès la virgule) seront affichés comme des entiers et que le style par défaut est le 1 car il permet d'afficher les matrices de façon compact. En effet, il est commun en Max-Plus de devoir manipuler et afficher de grosses matrices remplies d'éléments $\varepsilon$.

In [18]:
# Affichage classique façon Julia
mp_change_display(0)

J = MP([-Inf 0; 0 -Inf])

2×2 Max-Plus dense matrix:
  -Inf    0.0
   0.0   -Inf


In [19]:
# Affichage des 0 sous la forme de e
mp_change_display(2)

J

2×2 Max-Plus dense matrix:
  .   e
  e   .


In [20]:
# Affichage des -Inf sous la forme de ε
mp_change_display(3)

J

2×2 Max-Plus dense matrix:
  ε   0
  0   ε


In [21]:
# Affichage des -Inf sous la forme de ε et les 0 sous la forme de e
mp_change_display(4)

J

2×2 Max-Plus dense matrix:
  ε   e
  e   ε


Et finalement, le mode par défaut :

In [22]:
# Affichage des -Inf sous la forme d'un .
mp_change_display(1)

J

2×2 Max-Plus dense matrix:
  .   0
  0   .


## Opérateur Max-Plus $\oplus$

L'opérateur addition est redéfini par l'opérateur `max` de l'algèbre classique. Son symbole, pour le différencier de l'addition dans l'algèbre classique, est $\oplus$. Mais en Julia on gardera le symbole `+`. Cet opérateur est associatif, commutatif, a un élément neutre (noté $\varepsilon$) et est idempotent. $\forall a,b,c \in \mathbb{R}_{\varepsilon}:$

$$a \oplus b \triangleq \max(a,b)$$

In [23]:
a = MP(1); b = MP(3); c = MP(5);
(a, b, c)

(1, 3, 5)

In [24]:
a + b    # ≜ max(a, b) == max(1, 3) == 3

Max-Plus 3

####  $\oplus$ n'est pas inversible ni simplifiable

L'égalité suivante $a \oplus b = a \oplus c$ n'entraine pas $b = c$. Par contre on aura $a \oplus b = a$ si $a \geq b$ ou plus généralement $a \oplus b = a$ ou $b$.

#### Commutativité de $\oplus$

$$a \oplus b = b \oplus a$$
$$\triangleq$$
$$\max(a,b) = \max(b,a)$$

In [25]:
a + b == b + a

true

#### Associativité de $\oplus$

$$a \oplus b \oplus c = (a \oplus b) \oplus c = a \oplus (b \oplus c)$$

In [26]:
a + b + c == (a + b) + c == a + (b + c)

true

In [27]:
a + b + c # ≜ max(a, b, c) == max(1, 3, 5)

Max-Plus 5

#### Elément neutre $\varepsilon$ pour $\oplus$

$$a \oplus \varepsilon = \varepsilon \oplus a = a$$
$$\triangleq$$
$$\max(a,-\infty) = \max(-\infty,a) = a$$

In [28]:
a + ε == ε + a == a

true

Equivallent à :

In [29]:
a + mp0 == mp0 + a == a

true

In [30]:
(a, mp0, ε), (a + mp0, a + ε), (mp0 + a, ε + a)

((1, ., .), (1, 1), (1, 1))

Notons que `0` est neutre pour les nombres positifs :

In [31]:
a + 0 == 0 + a == a

true

In [32]:
a, 0, a + 0

(1, 0, 1)

#### $\oplus$ est idempotent

In [33]:
a + a    # ≜ max(a, a) == max(1, 1) == 1

Max-Plus 1

## Opérateur Max-Plus $\otimes$

L'opérateur multiplication est redéfini par l'opérateur addition qui est associatif, commutatif, a l'élément neutre $e$, l'élément absorbant $\varepsilon$ et est distributif sur $\oplus$.

In [34]:
a * b    # ≜ a + b == 1 + 3 == 4

Max-Plus 4

#### Commutativité de $\otimes$

$$a \otimes b = b \otimes a$$
$$\triangleq$$
$$a + b = b + a$$

In [35]:
a * b == b * a

true

#### Associativité de $\otimes$

$$a \otimes b \otimes c = (a \otimes b) \otimes c = a \otimes (b \otimes c)$$

In [36]:
a * b * c == (a * b) * c == a * (b * c)

true

In [37]:
a * b * c

Max-Plus 9

#### Elément neutre $e$ pour $\otimes$

$$a \otimes e = e \otimes a = a$$
$$\triangleq$$
$$a + 0 = 0 + a = a$$

In [38]:
a * mpe == mpe * a == a

true

Equivalent à :

In [39]:
a * mp1 == mp1 * a == a

true

#### Elément absorbant $\varepsilon$ pour $\otimes$

$$a \otimes \varepsilon = \varepsilon \otimes a = \varepsilon$$
$$\triangleq$$
$$a -\infty = -\infty + a = -\infty$$

In [40]:
a * ε == ε * a == ε

true

Equivalent à :

In [41]:
a * mp0 == mp0 * a == mp0

true

Par convention:

$$+\infty \otimes \varepsilon = \varepsilon \otimes +\infty = \varepsilon$$

In [42]:
mptop * mp0 # FIXME shall return mp0

Max-Plus .

#### $\otimes$ n'est pas idempotent

In [43]:
a * a    # ≜ a + a == 1 + 1 == 2

Max-Plus 2

### Distributivité de $\otimes$ sur $\oplus$

$$a \otimes (b \oplus c) = (a \otimes b) \oplus (a \otimes c)$$
$$(b \oplus c) \otimes a = (b \otimes a) \oplus (c \otimes a)$$

$$a \oplus b \otimes c = a + \max(a, b)$$
$$a \otimes c \oplus b \otimes c = \max(a+c,b+c)$$

In [44]:
(a + b) * c == (a * c) + (b * c)     # => max(a, b) + c == max(a + c, b + c) 

true

In [45]:
(a * c) + (b * c)

Max-Plus 8

### Opérateur puissance

En algèbre Max-Plus l'opérateur puissance se comporte comme une multiplication dans l'algèbre classique :

In [46]:
MP(2.0)^5   # ==> 2 * 5

Max-Plus 10

In [47]:
MP(2.0)^0   # ==> 2 * 0

Max-Plus 0

In [48]:
MP(2.0)^-1   # ==> 2 * -1

Max-Plus -2

## Vecteur colonnes, matrices denses et creuses Max-Plus

Ce que l'on vient de voir sur les sclaires est également applicable sur les matrices denses et pleines (sauf régression possible lors d'une mise à jour Julia).

### Construction de vecteur colonne Max-Plus

In [49]:
MP(1:5)

5-element Max-Plus vector:
  1
  2
  3
  4
  5


In [50]:
MP(1:0.5:3)

5-element Max-Plus vector:
    1
  1.5
    2
  2.5
    3


### Matrices denses Max-Plus

Comme pour les sclaires, la contamination des nombres Max-Plus sur les nombres Int64 et Float64 fonctionne également sur les éléments des matrices denses et creuses :

In [51]:
[MP(1) 2; 3.0 4]

2×2 Max-Plus dense matrix:
  1   2
  3   4


`MP(1)` de type `MP` a contaminé les nombres entiers *classiques* `2`, `3.0` et `4` en nombre `MP`.

Voici une autre façon plus élégente de le faire :

In [52]:
MP([1 2; 3 4])

2×2 Max-Plus dense matrix:
  1   2
  3   4


Autre exemple de contamination des nombres Max-Plus:

In [53]:
f = 3; a = MP(1)
[f       a
 f + f   a + a]

2×2 Max-Plus dense matrix:
  3   1
  6   1


`f + f` étant des `Int64` l'addition classique a été faite avant la promotion en nombre `MP`. Par contre, `a + a` étant des `MP` c'est l'addition (max, +) qui a été utilisée. Finallement tous les éléments de la matrices sont de type `MP`.

La contamination fonctionne également sur les matrices creuses.

### Matrices creuses Max-Plus

Une matrice creuse est une matrice contenant beaucoup de zéros. Sa structure interne est conçue pour ne pas garder en mémoire ces zéros (sauf en Julia s'ils sont explicitement donnés). En algèbre classique les zéros sont 0 (pour les entiers) ou 0.0 (réels) mais en Max-Plus ils ont pour valeur $-\infty$ et par conséquent une matrice creuse Max-Plus ne stocke pas les $\varepsilon$. 

Beaucoup d'exemples dans la nature peuvent être représenté par des matrices creuses plutôt que par des matrices pleines comme par exemple le réseau routier est un graphe où les routes sont les arcs et les carrefours sont les noeuds. Les graphes sont représentés en mémoire soit sous forme de liste d'ajacence soit sous forme de matrice. Comme en général un carrefour permet de rejoindre entre 2 à 4 routes et que par conséquent jamais il n'existera une ville où tous les carrefours seront lés les uns aux autres il est donc préférable d'utiliser une matrice creuse. Mais à cause des leur index, les algorithmes sur les matrices seront plus pénalisés en temps d'exécution que pour les matrices pleines.

Pour créer une matrice creuse Max-Plus, plusieurs choix:
- soit à partir d'une matrice creuse initialement vide, comme la fonction `mpzeros` que l'on verra plus tard.
- soit à partir d'une matrice pleine avec la fonction `SparseArrays.sparse` couplée avec le constructeur `MP`.
- soit à partir de trois vecteurs et la fonction `SparseArrays.sparse` couplée avec le constructeur `MP` : un vecteur des données à stocker et deux vecteurs indiquant les index de ces données dans la matrice.

### A partir d'une matrice pleine

In [54]:
using SparseArrays;
mp_change_display(0);

In [55]:
S = MP(sparse([1 2; 0 4]))

2×2 Max-Plus sparse matrix with 3 stored entries:
  [1, 1]  =  1.0
  [1, 2]  =  2.0
  [2, 2]  =  4.0

Ici le zéro de l'algèbre classique (vallant 0) a été supprimé par `SparseArrays.sparse` mais dans l'exemple suivant c'est le zéro de l'algébre Max-Plus ($\varepsilon$ vallant $-\infty$) qui a sera supprimé.

In [56]:
S = sparse(MP([1 2; -Inf 4]))

2×2 Max-Plus sparse matrix with 3 stored entries:
  [1, 1]  =  1.0
  [1, 2]  =  2.0
  [2, 2]  =  4.0

Le lecteur attentif aura apperçu que l'affichage est celui de Julia 1.5 même si Julia 1.6 est utilisé. En effet, avec Julia 1.6 l'affichage d'une matrice creuse se fait de la même manière qu'une matrice dense ce qui n'a pas de sens selon nous et l'ancien affichage est forcé mais uniquement pour les matrices creuses Max-Plus.

Pour rappel, a fonction `SparseArrays.findnz` retourne les éléments stockés `D` ainsi que leur indices `I` et `J` sous forme d'un triplet de vecteurs colonnes :

In [57]:
i,j,d = findnz(S)

([1, 1, 2], [1, 2, 2], MP[1.0, 2.0, 4.0])

### Construction explicite creuse

Tout comme `SparseArrays.findnz` retournant un triplet de vecteur colonnes `I`, `J` et `D`, la `SparseArrays.sparse` accepte ses mêmes paramètres. Mais les zéros explicites seront stockés :

In [58]:
S = MP(sparse([1; 2; 3], [1; 2; 3], [42; 0; 5]))

3×3 Max-Plus sparse matrix with 3 stored entries:
  [1, 1]  =  42.0
  [2, 2]  =  0.0
  [3, 3]  =  5.0

Ici le zéro de l'algèbre classique (vallant 0) n'a été supprimé par `SparseArrays.sparse` et dans l'exemple suivant c'est le zéro de l'algébre Max-Plus ($\varepsilon$ vallant $-\infty$) qui n'a pas été supprimé.

In [59]:
S = sparse([1; 2; 3], [1; 2; 3], MP([42; -Inf; 5]))

3×3 Max-Plus sparse matrix with 3 stored entries:
  [1, 1]  =  42.0
  [2, 2]  =  -Inf
  [3, 3]  =  5.0

Si vous ne souhaitez pas faire un `using SparseArrays` vous pouvez créer une matrice creuse depuis le constructeur `MP` qui gardera les zéros explicites :

In [60]:
S = MP([1; 2; 3], [1; 2; 3], MP([42; -Inf; 5]))

3×3 Max-Plus sparse matrix with 3 stored entries:
  [1, 1]  =  42.0
  [2, 2]  =  -Inf
  [3, 3]  =  5.0

### Conversion de matrices Max-Plus vers Min-Plus

Comme vu précédement pour les scalaire, on peut vouloir convertir les valeurs pour l'algèbre Min-Plus (les signes des $-\infty$ sont inversés) pour les matrices creuses ou pleines :

In [61]:
# Depuis une matrice creuse :
Z = mpzeros(2,2)
minplus(Z)

2×2 Max-Plus sparse matrix with 4 stored entries:
  [1, 1]  =  Inf
  [2, 1]  =  Inf
  [1, 2]  =  Inf
  [2, 2]  =  Inf

In [62]:
# Depuis une matrice pleine :
A = MP([4 0; 7 -Inf])
A, minplus(A)

(MP[4.0 0.0; 7.0 -Inf], MP[4.0 0.0; 7.0 Inf])

### Conversion de matrices Max-Plus vers algèbre classique

Comme vu précédement pour les scalaires, on peut vouloir convertir une matrice Max-Plus en valeurs de l'algèbre classique :

In [63]:
A = MP([4 0; 7 -Inf])
plustimes(A)

2×2 Matrix{Float64}:
 4.0    0.0
 7.0  -Inf

Fonctionne aussi pour les matrices creuses :

In [64]:
Z = mpzeros(2,2)
plustimes(Z)

2×2 SparseMatrixCSC{Float64, Int64} with 0 stored entries:
  ⋅    ⋅ 
  ⋅    ⋅ 

On peut vouloir passer d'une matrice creuse Max-Plus en matrice pleine  dans l'algèbre classique :

In [65]:
array(Z)

2×2 Matrix{Float64}:
 -Inf  -Inf
 -Inf  -Inf

### Conversion d'une matrice creuse en matrice pleine :

Les trois fonctions produisent le même résultat :

In [66]:
full(Z),  dense(Z),   Matrix(Z)

(MP[-Inf -Inf; -Inf -Inf], MP[-Inf -Inf; -Inf -Inf], MP[-Inf -Inf; -Inf -Inf])

## Construction de matrices Max-Plus usuelles

### Matrice dense d'identité

Par exemple de taille 2 $\times$ 2 :

$$\left[
\begin{array}{*{20}c}
e & \varepsilon \\
\varepsilon & e \\
\end{array}
\right]$$

Depuis Julia v1.0, la fonction `eye` n'existe plus et a été remplacée par `LinearAlgebra.I` mais cette boite à outils ajoute leur équivalent `mpeye` et `mpI` :

In [67]:
using LinearAlgebra
I

UniformScaling{Bool}
true*I

In [68]:
Matrix{MP}(I, 2, 2)

2×2 Max-Plus dense matrix:
   0.0   -Inf
  -Inf    0.0


In [69]:
Matrix{MP}(I, 2, 2)

2×2 Max-Plus dense matrix:
   0.0   -Inf
  -Inf    0.0


In [70]:
mpI

UniformScaling{MP}
0.0*I

In [71]:
Matrix(mpI, 2, 2)

2×2 Max-Plus dense matrix:
   0.0   -Inf
  -Inf    0.0


In [72]:
Matrix(mpI, 2, 2) == mpeye(2,2),   Matrix{MP}(I, 2, 2) == mpeye(2,2)

(true, true)

La fonction `mpeye` reste plus simple à taper :

In [73]:
J = mpeye(2,2)

2×2 Max-Plus dense matrix:
   0.0   -Inf
  -Inf    0.0


In [74]:
J = mpeye(2) # Equivalent à mpeye(2,2)

2×2 Max-Plus dense matrix:
   0.0   -Inf
  -Inf    0.0


Taille 3 $\times$ 2 :

In [75]:
J = mpeye(3,2)

3×2 Max-Plus dense matrix:
   0.0   -Inf
  -Inf    0.0
  -Inf   -Inf


### Matrices/Vecteur colonne denses remplies uniquement de $e$ :

Par exemple matrice de taille 2 $\times$ 2 :

$$\left[
\begin{array}{*{20}c}
e & e \\
e & e \\
\end{array}
\right]$$

In [76]:
O = mpones(2,2)

2×2 Max-Plus dense matrix:
  0.0   0.0
  0.0   0.0


Vecteur colonne de 2 éléments:

In [77]:
O = mpones(2) # /!\ N'est pas équivalent à mpones(2,2) /!\

2-element Max-Plus vector:
  0.0
  0.0


### Matrices creuses remplies de $\varepsilon$ :

$$\left[
\begin{array}{*{20}c}
\varepsilon & \varepsilon \\
\varepsilon & \varepsilon \\
\end{array}
\right]$$

In [78]:
Z = mpzeros(2,2)

2×2 Max-Plus sparse matrix with 0 stored entries

In [79]:
Z = mpzeros(2,3)

2×3 Max-Plus sparse matrix with 0 stored entries

In [80]:
Z = mpzeros(2)

2-element SparseVector{MP, Int64} with 0 stored entries

On remarquera que ces matrices sont vides. En effet, ils correspondent aux 0 éliminés des matrices creuses en algèbre classique. Une matrice creuse Max-Plus ne stocke pas les nombres Max-Plus $-\infty$ (**note:** enfin jusqu'à Julia > 1.3 car les versions précédentes avaient un bogue elles confondaient 0 et zero(T) avec T template de type MP).

### Matrices denses remplies de $\varepsilon$ :

Il faut utiliser la fonction `full` ou bien la fonction synonyme `dense`.

In [81]:
Z = full(mpzeros(2,2))

2×2 Max-Plus dense matrix:
  -Inf   -Inf
  -Inf   -Inf


## Opérateur éléments par éléments sur les matrices

Julia permet d'itérer sur les éléments d'un tableau, matrice, vecteur et appliquer une opération sur chacun d'eux. Par exemple :

$$4 \oplus \left[
\begin{array}{*{20}c}
2 \\
8\\
\end{array}
\right] = \left[
\begin{array}{*{20}c}
4 \oplus 2 \\
4 \oplus 8\\
\end{array}
\right] = \left[
\begin{array}{*{20}c}
4 \\
8\\
\end{array}
\right]$$

In [82]:
A = MP([1.0 2; 3 4])

2×2 Max-Plus dense matrix:
  1.0   2.0
  3.0   4.0


On applique la fonction max(2, ) sur chacun des éléments qui seront contaminés en nombre Max-Plus :

In [83]:
2 .+ A

2×2 Max-Plus dense matrix:
  2.0   2.0
  3.0   4.0


In [84]:
A .+ 2.0

2×2 Max-Plus dense matrix:
  2.0   2.0
  3.0   4.0


On applique la fonction +(2, ) sur chacun des éléments 

In [85]:
2 .* A

2×2 Max-Plus dense matrix:
  3.0   4.0
  5.0   6.0


In [86]:
A .* 2.0

2×2 Max-Plus dense matrix:
  3.0   4.0
  5.0   6.0


## Addition et produit matriciel

Les matrices peuvent être de type Max-Plus. L'addition et le produit matriciel Max-Plus correspond à l'addition et au produit matriciel avec les opérateurs $+$ et $\times$ surchargés.

### Addition matricielle

$$\begin{bmatrix}
1 & 6 \\
8 & 3
\end{bmatrix} \oplus \begin{bmatrix}
2 & 5 \\
3 & 3
\end{bmatrix} = \begin{bmatrix}
1 \oplus 2 & 6 \oplus 5 \\
8 \oplus 3 & 3 \oplus 3
\end{bmatrix} = \begin{bmatrix}
2 & 6 \\
8 & 3
\end{bmatrix}$$

In [87]:
MP([1 6; 8 3]) + MP([2 5; 3 3])

2×2 Max-Plus dense matrix:
  2.0   6.0
  8.0   3.0


### Produit matriciel

$$A=\begin{bmatrix}
4 & 3 \\
7 & -\infty
\end{bmatrix}\;,$$

$$A \otimes A = \begin{bmatrix}
4 \otimes 4 \oplus 3 \otimes7 & 4 \otimes 3 \oplus 3 \otimes -\infty \\
7 \otimes 4 \oplus -\infty \otimes 7 & 7 \otimes 3 \oplus -\infty \otimes -\infty
\end{bmatrix}\; = \begin{bmatrix}
10 & 7 \\
11 & 10
\end{bmatrix}\; = A^2.$$

In [88]:
A = MP([4 3; 7 -Inf])
A * A

2×2 Max-Plus dense matrix:
  10.0    7.0
  11.0   10.0


In [89]:
A * A == A^2

true

Fonctionne également sur les matrices creuses :

In [90]:
A * sparse(A) == sparse(A) * A == sparse(A) * sparse(A)

true

S'applique également aux matrices :

In [91]:
A^5

2×2 Max-Plus dense matrix:
  24.0   23.0
  27.0   24.0


In [92]:
A^0

2×2 Max-Plus dense matrix:
   0.0   -Inf
  -Inf    0.0


S'applique également aux vecteurs colonne et vecteurs ligne :

In [93]:
MP([2 0; mp0 5]) * MP([2; 8])

2-element Max-Plus vector:
   8.0
  13.0


In [94]:
MP([2 8]) * MP([2 0; mp0 5])

1×2 Max-Plus dense matrix:
  4.0   13.0


Vérifions que la matrice identité $I$ est bien neutre :

$$ A \otimes I = I \otimes A == A$$

In [95]:
A = MP([4 3; 7 -Inf])
A * mpeye(2,2) == mpeye(2,2) * A == A

true

Vérifions la matrice zéro :

In [96]:
A * mpzeros(2,2) == mpzeros(2,2) * A == mpzeros(2,2)

true

In [97]:
A + mpzeros(2,2) == mpzeros(2,2) + A == A

true

## Affichage des matrices Max-Plus en LaTeX

A partir d'une matrice Max-Plus, on peut générer le code $\LaTeX$ grâce à la fonction `LaTeX` ou via la fonction `show` avec l'argument `MIME"text/latex"`. La fonction `mp_change_display` modifie en conséquence les éléments neutres et absorbants du code LaTeX généré.

In [98]:
mp_change_display(0)
LaTeX(stdout, mpeye(2,2))

\left[
\begin{array}{*{20}c}
0 & -\infty \\
-\infty & 0 \\
\end{array}
\right]


Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
0 & -\infty \\
-\infty & 0 \\
\end{array}
\right]$$

Alors que :

In [99]:
mp_change_display(1)
LaTeX(stdout, mpeye(2,2))

\left[
\begin{array}{*{20}c}
0 & . \\
. & 0 \\
\end{array}
\right]


Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
0 & . \\
. & 0 \\
\end{array}
\right]
$$

Fonctionne avec les matrices creuses :

In [100]:
mp_change_display(1)
LaTeX(stdout, mpzeros(2,2))

\left[
\begin{array}{*{20}c}
. & . \\
. & . \\
\end{array}
\right]


Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
. & . \\
. & . \\
\end{array}
\right]$$

## Trace d'une matrice Max-Plus

La trace est la somme Max-Plus des éléments diagonaux.

In [101]:
A = MP([4 3; 7 -Inf])
mptrace(A)

Max-Plus 4

In [102]:
mptrace(A) == A[1,1] + A[2,2]

true

## Norme d'une matrice

In [103]:
mpnorm(A)

Max-Plus Inf

In [104]:
mpnorm(mpeye(2,2))

Max-Plus Inf

## Inverse d'une matrice Max-Plus

TODO

## Valeurs propres d'une matrice Max-Plus $A v = \lambda v$

Le spectre d'une matrice est l'ensemble de ses valeurs propres. Une matrice carrée $A \in \mathbb{R}_{\varepsilon}^{n \times n}$ a une valeur propre s'il existe un nombre réel $\lambda \in \mathbb{R}^{n}$ et un vecteur $v \in \mathbb{R}_{\varepsilon}^{n}$ si :

$$A v = \lambda v$$

$v$ est appelé vecteur propre et $\lambda$ valeur propre. Dans le boite à outils Max-Plus de Scilab, ils sont obtenus soit par les fonctions `karp` ou `howard`. Dans cette boite à outils Julia seul `howard` est donné car son algorithme est plus rapide (linéaire au nombre d'arcs) que l'algorithme de `karp` ($O(mn)$).

In [105]:
A = sparse(MP([1 2; 3 4]))
λ,v = howard(A)

(MP[4, 4], MP[2, 4])

In [106]:
A * v == λ[1] * v

true

In [107]:
A * v == λ[2] * v

true

Ces éléments spectraux donnent le comportement asymptotique des systèmes dynamiques Max-Plus :

In [108]:
x = [mp1; mp0]

2-element Max-Plus vector:
  0
  .


In [109]:
[x A*x A^2*x A^3*x]

2×4 Max-Plus dense matrix:
  0   1   5    9
  .   3   7   11


## Résolution d'équations linéaires Max-Plus $x = Ax \oplus b$

Soit $A \in \mathbb{R}_{\varepsilon}^{n \times n}$ une matrice Max-plus carrée et $b \in \mathbb{R}_{\varepsilon}^{n}$ un vecteur colonne. La solution de $x = Ax \oplus b$ est donnée par :
$$x = A^* b$$

Où :

$$A^+ \triangleq A^1 \oplus A^2 \oplus A^3 \oplus\;...$$
$$A^* \triangleq A^0 \oplus A^+$$

$A^0$ n'est d'autre que la matrice identité Max-Plus. $A^+$ est calculé par la fonction `mpplus` et $A^*$ est calculé par la fonction `mpstar`. La solution de l'équation est donnée par la fonction `mpastarb`.

In [110]:
mpstar(MP(2))

Max-Plus Inf

In [111]:
mpstar(MP(-2))

Max-Plus 0

In [112]:
A = MP([-3 -2; -1 0])
mpplus(A)

2×2 Max-Plus dense matrix:
  -3   -2
  -1    0


In [113]:
mpstar(A)

2×2 Max-Plus dense matrix:
   0   -2
  -1    0


In [114]:
mpplus(A) == A * mpstar(A)

true

In [115]:
# TODO for sparse matrix
# mpstar(mpzeros(2,2)) == mpstar(full(mpzeros(2,2)))

In [116]:
b = MP([mp0; mp1])

2-element Max-Plus vector:
  .
  0


In [117]:
x = mpastarb(A, b)

2-element Max-Plus vector:
  -2
   0


In [118]:
x == A * x + b

true